In [4]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
#Importing csv data
states_data = pd.read_csv('4.18states.csv')
counties = pd.read_csv('abridged_couties.csv')
confirmed = pd.read_csv('time_series_covid19_confirmed_US.csv')
deaths = pd.read_csv('time_series_covid19_deaths_US.csv')

In [57]:
#Cleaning Data to answer the following questions
#We will first create tables only where California is concerned
ca_data = states_data[states_data["Province_State"] == 'California']
ca_counties = counties[counties["State"] == 'California'][["countyFIPS",  "lat", "lon", "PopulationEstimate2018", "PopTotalMale2017","PopTotalFemale2017", "PopulationDensityperSqMile2010", "MedicareEnrollment,AgedTot2017", "SVIPercentile"]]
ca_deaths = deaths[deaths["Province_State"] == 'California']

#We only care about the total deaths, which is found in column 4/18/20
ca_deaths = ca_deaths[["FIPS", "4/18/20"]]
ca_deaths.columns = ["FIPS", "deaths"]

confirmed_ca = confirmed[confirmed["Province_State"] == "California"]
#We only care about total confirmed cases, which is also the column 4/18/20
confirmed_ca = confirmed_ca[["FIPS", "4/18/20"]]
confirmed_ca.columns = ["FIPS", "confirmed"]

#convert String FIPS to float so we can use it in conjunction with other tables
ca_counties["countyFIPS"] = ca_counties["countyFIPS"].apply(lambda x: float(x))
#Join deaths and confirmed columns into counties table using FIPS. Remove counties without sufficient death data.
ca_counties = ca_counties.join(other = ca_deaths, lsuffix = "countyFIPS", rsuffix = "FIPS")
ca_counties = ca_counties.join(other = confirmed_ca, lsuffix = "countyFIPS", rsuffix = "FIPS")
ca_counties = ca_counties.fillna(-1)
ca_counties = ca_counties[ca_counties["deaths"] >= 0]
ca_counties = ca_counties[ca_counties["confirmed"] >= 0]
ca_counties = ca_counties.drop(["FIPSFIPS", "FIPScountyFIPS"], axis = 1)
ca_counties

,countyFIPS,lat,lon,PopulationEstimate2018,PopTotalMale2017,PopTotalFemale2017,PopulationDensityperSqMile2010,"MedicareEnrollment,AgedTot2017",SVIPercentile,deaths,confirmed
191,6015.0,41.760269,-123.903650,27828.0,14784.0,12686.0,28.4,4740.0,0.9713,41.0,1114.0
192,6017.0,38.780757,-120.528545,190678.0,94342.0,94645.0,106.0,36864.0,0.1838,0.0,2.0
193,6019.0,36.753678,-119.641036,994400.0,493547.0,495708.0,156.2,115579.0,0.9611,0.0,7.0
194,6021.0,39.605124,-122.391279,28047.0,14235.0,13859.0,21.4,4613.0,0.8895,0.0,16.0
195,6023.0,40.707453,-123.875444,136373.0,68119.0,68635.0,37.7,22746.0,0.7605,0.0,12.0
196,6025.0,33.044716,-115.375141,181827.0,93743.0,89087.0,41.8,25375.0,0.9978,0.0,3.0
197,6027.0,36.516350,-117.418497,17987.0,9077.0,8949.0,1.8,4036.0,0.5701,19.0,685.0
198,6029.0,35.341629,-118.728692,896764.0,457994.0,435125.0,103.3,91664.0,0.9659,0.0,2.0
199,6031.0,36.066444,-119.805537,151366.0,82356.0,67745.0,110.1,13838.0,0.9707,0.0,36.0
200,6033.0,39.109082,-122.762571,64382.0,31990.0,32256.0,51.5,13415.0,0.9178,7.0,311.0
